# Master jupyter notebook for LANL - SlimBros Team

Correctly predicting earthquakes is very important for preventing deaths and damage to infrastructure. In this competition we try to predict time left to the next laboratory earthquake based on seismic signal data. Training data represents one huge signal, but in test data we have many separate chunks, for each of which we need to predict time to failure.

## Preliminaries
Let's import everything we need:

In [1]:
import os
import gc
import eli5
import csv
import dill
import time
import random
import datetime
import warnings
import feather

import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import lightgbm as lgb
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from catboost import CatBoostRegressor
from sklearn import svm, neighbors, linear_model
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.svm import NuSVR, SVR
from sklearn.neighbors import NearestNeighbors
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GridSearchCV, cross_val_score, ParameterGrid, train_test_split
from utils import generate_segment_start_ids, compare_methods
from features import gpi, create_all_features_extended
from features import gpi_new, gpii_new, gpiii_new

#Configure the environment
%matplotlib inline
pd.options.display.precision = 15
warnings.filterwarnings('ignore')
random.seed(1013)

Load/compute the necessary features

In [2]:
compute_features = False 
# The computed features are saved in an hdf file along with the time_to_failure to 
# save the time spend reading the training data and the feature computation
#train_data_format = 'csv'
train_data_format = 'feather'

## Training data

In [3]:
def load_train_data(file_format):
    """Load the training dataset."""
    print(f"Loading data from {file_format} file:", end="")
    if file_format.lower() == 'feather':
        train_df = feather.read_dataframe('../input/train.feather')
    else:
        train_df = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16,
                                                            'time_to_failure': np.float32})
        feather.write_dataframe(train_df, '../input/train.feather')
    print("Done")
    return train_df

In [4]:
train = load_train_data(train_data_format)

Loading data from feather file:Done


In [5]:
time_to_failure_delta = np.diff(train['time_to_failure'])
init_times = np.where(time_to_failure_delta > 5)[0].tolist()
print(f"There are {len(init_times)} quakes on the training set.")
init_times = [0] + init_times

There are 16 quakes on the training set.


In [6]:
d = {'start_idx': init_times, 'end_idx': init_times[1:] + [len(time_to_failure_delta)]}
quakes = pd.DataFrame(data=d)
quakes.insert(2, 'valid', True)

Let's discard any quake that looks weird

## Feature generation
- Usual aggregations: mean, std, min and max;
- Average difference between the consequitive values in absolute and percent values;
- Absolute min and max vallues;
- Aforementioned aggregations for first and last 10000 and 50000 values - I think these data should be useful;
- Max value to min value and their differencem also count of values bigger that 500 (arbitrary threshold);
- Quantile features from this kernel: https://www.kaggle.com/andrekos/basic-feature-benchmark-with-quantiles
- Trend features from this kernel: https://www.kaggle.com/jsaguiar/baseline-with-abs-and-trend-features
- Rolling features from this kernel: https://www.kaggle.com/wimwim/rolling-quantiles

In [7]:
saved_files_present = (os.path.isfile('../tmp_results/Xv2_tr.hdf') and 
                       os.path.isfile('../tmp_results/Xv2_test.hdf') and 
                       os.path.isfile('../tmp_results/yv2_tr.hdf') )

In [8]:
#dill.dump_session('notebook_env.db')
#dill.load_session('notebook_env.db')

In [9]:
saved_files_present

True

In [10]:
if (not compute_features) and saved_files_present:
    print(f"Reading hdf files:", end="")
    X_tr = pd.read_hdf('../tmp_results/Xv2_tr.hdf', 'data')
    X_test = pd.read_hdf('../tmp_results/Xv2_test.hdf', 'data')
    y_tr = pd.read_hdf('../tmp_results/yv2_tr.hdf', 'data')  
    print("Done")
else:
    fs = 4000000 #Sampling frequency of the raw signal

    #Compute features for the training data
    segment_size = 150000
    segment_start_ids = generate_segment_start_ids('uniform_no_jump', segment_size, train)
    X_tr = pd.DataFrame(index=range(len(segment_start_ids)), dtype=np.float64)
    y_tr = pd.DataFrame(index=range(len(segment_start_ids)), dtype=np.float64, columns=['time_to_failure'])
    for idx in tqdm_notebook(range(len(segment_start_ids))):        
        seg_id = segment_start_ids[idx]
        seg = train.iloc[seg_id:seg_id + segment_size]
        create_all_features_extended(idx, seg, X_tr, fs)
        y_tr.loc[idx, 'time_to_failure'] = seg['time_to_failure'].values[-1]
    # Sanity check
    means_dict = {}
    for col in X_tr.columns:
        if X_tr[col].isnull().any():
            print(col)
            mean_value = X_tr.loc[X_tr[col] != -np.inf, col].mean()
            X_tr.loc[X_tr[col] == -np.inf, col] = mean_value
            X_tr[col] = X_tr[col].fillna(mean_value)
            means_dict[col] = mean_value

    #Compute features for the test data
    submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
    X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)
    for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
        seg = pd.read_csv('../input/test/' + seg_id + '.csv')
        create_all_features_extended(seg_id, seg, X_test, fs)

    # Sanity check
    for col in X_test.columns:
        if X_test[col].isnull().any():
            X_test.loc[X_test[col] == -np.inf, col] = means_dict[col]
            X_test[col] = X_test[col].fillna(means_dict[col])
            
    X_tr.to_hdf('../tmp_results/Xv2_tr.hdf', 'data')
    X_test.to_hdf('../tmp_results/Xv2_test.hdf', 'data')
    y_tr.to_hdf('../tmp_results/yv2_tr.hdf', 'data')
    
    del segment_start_ids
    del means_dict
    del submission
    
    print("Done")

Reading hdf files:Done


## Scale data

In [11]:
alldata = pd.concat([X_tr, X_test])
scaler = StandardScaler()
alldata = pd.DataFrame(scaler.fit_transform(alldata), columns=alldata.columns)
X_train_scaled = alldata[:X_tr.shape[0]]
X_test_scaled = alldata[X_tr.shape[0]:]

## Building models

In [12]:
def train_model(X, X_test, y, folds, params=None, model_type='lgb',
                model=None, show_scatter=False, force_positive=False):

    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    n_fold = folds.get_n_splits()
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        if model_type == 'nn':
            dropout = params['dropout']
            num_layers = params['num_layers']
            num_neurons = params['num_neurons']
            activation_function = params['activation_function']
            model = tf.keras.Sequential()
            model.add(tf.keras.layers.Dense(1024, input_dim=216, activation=activation_function))
            model.add(tf.keras.layers.BatchNormalization())
            model.add(tf.keras.layers.Dropout(dropout))
            for l in range(num_layers):
                model.add(tf.keras.layers.Dense(num_neurons, activation=activation_function))
                model.add(tf.keras.layers.BatchNormalization())
                model.add(tf.keras.layers.Dropout(dropout))
            model.add(tf.keras.layers.Dense(1))
            model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
            EPOCHS = 1000
            early_stop = tf.keras.callbacks.EarlyStopping(monitor='mean_absolute_error', patience=100)

            history = model.fit(
                X_train,
                y_train,
                epochs=EPOCHS,
                validation_data = (X_valid, y_valid), 
                verbose=0,
                callbacks=[early_stop, PrintDot()])
            hist = pd.DataFrame(history.history)
            val_score = hist['val_mean_absolute_error'].iloc[-1]
            print(f'val_score={val_score}')
            plot_history(history)
        
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)
          
            y_pred = model.predict(X_test).reshape(-1,)

            score = mean_absolute_error(y_valid, y_pred_valid)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 50000, n_jobs = -1)
            model.fit(X_train, y_train, 
                      eval_set=[(X_train, y_train), (X_valid, y_valid)],
                      eval_metric='mae',
                      verbose=10000,
                      early_stopping_rounds=2000)
            
            y_pred_valid = model.predict(X_valid)
            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)
          
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data,
                              num_boost_round=20000,
                              evals=watchlist,
                              early_stopping_rounds=200,
                              verbose_eval=500,
                              params=params)

            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns),
                                         ntree_limit=model.best_ntree_limit)
            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)
          
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns),
                                   ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)
                
            score = mean_absolute_error(y_valid, y_pred_valid)
            y_pred = model.predict(X_test).reshape(-1,)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
            print('')
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric='MAE', task_type='GPU', **params)
            model.fit(X_train,
                      y_train,
                      eval_set=(X_valid, y_valid),
                      cat_features=[],
                      use_best_model=True,
                      verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)

            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)

        if model_type == 'gpi':
            y_pred_valid = gpi(X_valid, activation=params['activation']).values
            y_pred = gpi(X_test, activation=params['activation']).values
            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)

        if model_type == 'gpi_new':
            y_pred_valid = gpi_new(X_valid, activation=params['activation']).values
            y_pred = gpi_new(X_test, activation=params['activation']).values
            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)
            
        if model_type == 'gpii_new':
            y_pred_valid = gpii_new(X_valid, activation=params['activation']).values
            y_pred = gpii_new(X_test, activation=params['activation']).values
            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)
                
        if model_type == 'gpiii_new':
            y_pred_valid = gpiii_new(X_valid, activation=params['activation']).values
            y_pred = gpiii_new(X_test, activation=params['activation']).values
            if force_positive:
                y_pred_valid = y_pred_valid.clip(min=0)
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_absolute_error(y_valid, y_pred_valid))
        
        prediction += y_pred    
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance['feature'] = X.columns
            fold_importance['importance'] = model.feature_importances_
            fold_importance['fold'] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    if force_positive:
        prediction = prediction.clip(min=0)    
    
    if show_scatter:
        fig, axis = plt.subplots(1, 2, figsize=(12,5))
        ax1, ax2 = axis
        ax1.set_xlabel('actual')
        ax1.set_ylabel('predicted')
        ax2.set_xlabel('train index')
        ax2.set_ylabel('time to failure')
        
        ax1.scatter(y, oof, color='brown')
        ax1.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)], color='blue')

        ax2.plot(y, color='blue', label='y_train')
        ax2.plot(oof, color='orange')
    
    print(f'CV mean score: {np.mean(scores):.4f}, std: {np.std(scores):.4f}.')
    
    if model_type == 'lgb':
        feature_importance['importance'] /= n_fold
        return oof, prediction, scores, feature_importance
    else:
        return oof, prediction, scores

In [13]:
n_fold = 5
folds_models = KFold(n_splits=n_fold, shuffle=True, random_state=11)

Let's try a few different models and submit the one with the best validation score. The predicted values in the following plots are using a out-of-fold scheme.

### LGBM (Gradient Boosting)

In [14]:
best_params_lgb_huber = {
    "objective": "huber",
    "boosting": "gbdt",
    "verbosity": -1,
    "num_leaves": 12,
    "min_data_in_leaf": 40,
    "max_depth": 8,
    "learning_rate": 0.005,
    "bagging_freq": 4,
    "bagging_fraction": 0.6,
    "bagging_seed": 11,
    "random_seed": 19,
    "metric": "mae",
    "reg_alpha": 0.47777777777777775,
    "reg_lambda": 0.47777777777777775
}

In [15]:
best_params_lgb_gamma = {
    'num_leaves': 36,
    'min_data_in_leaf': 10,
    'max_depth': 12,
    'learning_rate': 0.005,
    'bagging_freq': 3,
    'bagging_fraction': 0.7166666666666667,
    'reg_alpha': 0.28888888888888886,
    'reg_lambda': 0.95,
    'objective': 'gamma',
    'boosting': 'gbdt',
    'verbosity': -1,
    'random_seed': 19,
    'metric': 'mae',
    'bagging_seed': 11
}

In [16]:
best_params_lgb_fair = {
    'num_leaves': 8,
    'min_data_in_leaf': 60,
    'max_depth': 3,
    'learning_rate': 0.01,
    'bagging_freq': 5,
    'bagging_fraction': 0.6388888888888888,
    'reg_alpha': 0.19444444444444445,
    'reg_lambda': 0.95,
    'objective': 'fair',
    'boosting': 'gbdt',
    'verbosity': -1,
    'random_seed': 19,
    'metric': 'mae',
    'bagging_seed': 11
}

In [17]:
best_params_lgb_mae = {
    "num_leaves": 8,
    "min_data_in_leaf": 100,
    "max_depth": 16,
    "learning_rate": 0.05,
    "bagging_freq": 3,
    "bagging_fraction": 0.6777777777777778,
    "reg_alpha": 0.19444444444444445,
    "reg_lambda": 0.1,
    "objective": "mae",
    "boosting": "gbdt",
    "verbosity": -1,
    "random_seed": 19,
    "metric": "mae",
    "bagging_seed": 11}

In [18]:
best_params_lgb_regression = {
    "num_leaves": 80,
    "min_data_in_leaf": 100,
    "max_depth": 8,
    "learning_rate": 0.01,
    "bagging_freq": 5,
    "bagging_fraction": 0.7166666666666667,
    "reg_alpha": 0.28888888888888886,
    "reg_lambda": 0.6666666666666666,
    "objective": "regression",
    "boosting": "gbdt",
    "verbosity": -1,
    "random_seed": 19,
    "metric": "mae",
    "bagging_seed": 11
}

In [24]:
best_params_lgb_hyperopt = {
    "bagging_fraction": 0.84,
    "boosting": "gbdt",
    "feature_fraction": 0.55,
    "lambda_l1": 2.0256449252260045,
    "lambda_l2": 0.5147476786636518,
    "learning_rate": 0.0662641996692177,
    "max_bin": 131,
    "max_depth": 24,
    "metric": "MAE",
    "min_data_in_bin": 48,
    "min_data_in_leaf": 234,
    "min_gain_to_split": 1.81,
    "num_leaves": 981,
    "objective": "fair",
    "subsample": 0.9630227452314452
}

In [19]:
params_lgb = best_params_lgb_huber

In [20]:
oof_lgb, prediction_lgb, scores_lgb, feature_importance_lgb = train_model(X=X_train_scaled,
                                                                          X_test=X_test_scaled,
                                                                          y=y_tr,
                                                                          folds=folds_models,
                                                                          params=params_lgb,
                                                                          model_type='lgb',
                                                                          show_scatter=False)

Fold 0 started at Fri May 10 17:26:11 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[5148]	training's l1: 1.22941	valid_1's l1: 2.01089
Fold 1 started at Fri May 10 17:28:00 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[2716]	training's l1: 1.57238	valid_1's l1: 2.00756
Fold 2 started at Fri May 10 17:29:13 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[2844]	training's l1: 1.52788	valid_1's l1: 2.09075
Fold 3 started at Fri May 10 17:30:29 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[2804]	training's l1: 1.58911	valid_1's l1: 1.85877
Fold 4 started at Fri May 10 17:31:43 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[2608]	training's l1: 1.57541	valid_1's l1: 2.05721
CV mean score: 2.0050, std: 0.

In [25]:
oof_lgb, prediction_lgb, scores_lgb, feature_importance_lgb = train_model(X=X_train_scaled,
                                                                          X_test=X_test_scaled,
                                                                          y=y_tr,
                                                                          folds=folds_models,
                                                                          params=best_params_lgb_hyperopt,
                                                                          model_type='lgb',
                                                                          show_scatter=False)

Fold 0 started at Fri May 10 17:35:41 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[65]	training's l1: 1.70334	valid_1's l1: 2.03212
Fold 1 started at Fri May 10 17:35:45 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[44]	training's l1: 1.79251	valid_1's l1: 2.03055
Fold 2 started at Fri May 10 17:35:49 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[81]	training's l1: 1.63362	valid_1's l1: 2.08423
Fold 3 started at Fri May 10 17:35:53 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[52]	training's l1: 1.79384	valid_1's l1: 1.88475
Fold 4 started at Fri May 10 17:35:57 2019
Training until validation scores don't improve for 2000 rounds.
Early stopping, best iteration is:
[58]	training's l1: 1.7284	valid_1's l1: 2.04483
CV mean score: 2.0153, std: 0.0681.


In [ ]:
cols = feature_importance_lgb[["feature", "importance"]].groupby("feature").mean().sort_values(
    by="importance", ascending=False)[:50].index
best_features = feature_importance_lgb.loc[feature_importance_lgb.feature.isin(cols)]
plt.figure(figsize=(16, 12));
sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
plt.title('LGB Features (avg over folds)');

### XGBoost (Gradient Boosting)

In [ ]:
params_xgb = {
    'eta': 0.01, #Andrew uses 0.03
    'max_depth': 6, #Andrew uses 10
    'subsample': 0.5, #Andrew uses 0.9
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'silent': True,
    'nthread': 32
} # CV mean score: 2.0801, std: 0.0711.
oof_xgb, prediction_xgb, scores_xgb = train_model(X=X_train_scaled,
                                                  X_test=X_test_scaled,
                                                  y=y_tr,
                                                  folds=folds_models,
                                                  params=params_xgb,
                                                  model_type='xgb',
                                                  show_scatter=False)

### SVM
Similar to NuSVC, for regression, uses a parameter nu to control the number of support vectors. However, unlike NuSVC, where nu replaces C, here nu replaces the parameter epsilon of epsilon-SVR.

In [ ]:
#model = NuSVR(gamma='scale', nu=0.9, C=10.0, tol=0.01) #original values
model = NuSVR(gamma='scale', nu=0.63, C=0.4469387755102041, tol=0.01)
oof_svr, prediction_svr, scores_svr = train_model(X=X_train_scaled,
                                      X_test=X_test_scaled,
                                      y=y_tr,
                                      folds=folds_models,
                                      params=None,
                                      model_type='sklearn',
                                      model=model,
                                      show_scatter=False)
del model

### Catboost

In [ ]:
params_cat = {
    'loss_function':'MAE'
}
oof_cat, prediction_cat, scores_cat = train_model(X=X_train_scaled,
                                                  X_test=X_test_scaled,
                                                  y=y_tr,
                                                  folds=folds_models,
                                                  params=params_cat,
                                                  model_type='cat',
                                                  show_scatter=False)

### Kernel Ridge
This model combines regularized linear regression with a given kernel (radial basis in this case).

In [ ]:
#model = KernelRidge(kernel='rbf', alpha=0.1, gamma=0.01) #Original parameters
model = KernelRidge(kernel='rbf', alpha=2.4497346938775513, gamma=0.0018461224489795917)
oof_r, prediction_r, scores_r = train_model(X=X_train_scaled,
                                            X_test=X_test_scaled,
                                            y=y_tr,
                                            folds=folds_models,
                                            params=None,
                                            model_type='sklearn',
                                            model=model,
                                            show_scatter=False)
del model

### Genetic Program Models

In [ ]:
params_gpi = {
    'activation':'atan'
}
oof_gpi, prediction_gpi, scores_gpi = train_model(X=X_train_scaled,
                                                  X_test=X_test_scaled,
                                                  y=y_tr,
                                                  folds=folds_models,
                                                  model_type='gpi',
                                                  params=params_gpi,
                                                  show_scatter=False)

In [ ]:
params_gpi_new = {
    'activation':'atan'
}
oof_gpi_new, prediction_gpi_new, scores_gpi_new = train_model(X_train_scaled,
                                                              X_test_scaled,
                                                              y_tr,
                                                              params=params_gpi_new,
                                                              folds=folds_models,
                                                              model_type='gpi_new')

In [ ]:
params_gpii_new = {
    'activation':'atan'
}
oof_gpii_new, prediction_gpii_new, scores_gpii_new = train_model(X_train_scaled,
                                                                 X_test_scaled,
                                                                 y_tr,
                                                                 params=params_gpii_new,
                                                                 folds=folds_models,
                                                                 model_type='gpii_new')

In [ ]:
params_gpiii_new = {
    'activation':'atan'
}
oof_gpiii_new, prediction_gpiii_new, scores_gpiii_new = train_model(X_train_scaled,
                                                                    X_test_scaled,
                                                                    y_tr,
                                                                    params=params_gpiii_new,
                                                                    folds=folds_models,
                                                                    model_type='gpiii_new')

### Random Forest
This regressor fits many decision trees with different subsets of the original data and average the predictions between them.

In [ ]:
params_rf = {
    'max_depth': 12, #8,
    'max_features': 'log2', #'auto',
    'min_samples_leaf': 2,
    'min_samples_split': 4 #6
} #CV mean score: 2.0448, std: 0.0793.
model = RandomForestRegressor(criterion='mae', n_estimators=200, n_jobs=-1, **params_rf)
oof_rf, prediction_rf, scores_rf = train_model(X=X_train_scaled,
                                               X_test=X_test_scaled,
                                               y=y_tr,
                                               folds=folds_models,
                                               params=params_rf,
                                               model_type='sklearn',
                                               model=model,
                                               show_scatter=False)
del model

### Extremely Randomized Trees


In [ ]:
params_ex = {
    'max_depth': 12,
    'max_features': 'sqrt',
    'min_samples_leaf': 2,
    'min_samples_split': 4
}
model = ExtraTreesRegressor(criterion='mae', n_estimators=200, n_jobs=-1, **params_ex)
oof_ex, prediction_ex, scores_ex = train_model(X=X_train_scaled,
                                               X_test=X_test_scaled,
                                               y=y_tr,
                                               folds=folds_models,
                                               params=params_ex,
                                               model_type='sklearn',
                                               model=model,
                                               show_scatter=False)
del model

### Ada Boost

In [ ]:
params_ada = {
    'learning_rate': 0.001
}
base = KernelRidge(kernel='rbf', alpha=2.4497346938775513, gamma=0.0018461224489795917)
model = AdaBoostRegressor(base_estimator=base, n_estimators=100, **params_ada)
oof_ada, prediction_ada, scores_ada = train_model(X=X_train_scaled,
                                                  X_test=X_test_scaled,
                                                  y=y_tr,
                                                  folds=folds_models,
                                                  params=params_ada,
                                                  model_type='sklearn',
                                                  model=model,
                                                  show_scatter=False)
del base
del model

### KNN

In [ ]:
#model = neighbors.KNeighborsRegressor()
#parameter_grid = {'n_neighbors': [10, 20, 25, 30, 35, 50]}
#grid_search = GridSearchCV(model, param_grid=parameter_grid,
#                           cv=folds_models,
#                           scoring='neg_mean_absolute_error',
#                           n_jobs=-1)
#grid_search.fit(X_train_scaled, y_tr)
#print('Best score: {}'.format(grid_search.best_score_))
#print('Best parameters: {}'.format(grid_search.best_params_))
#model = neighbors.KNeighborsRegressor(**grid_search.best_params_)
params_knn = {
    'n_neighbors': 10
}
model = neighbors.KNeighborsRegressor(n_neighbors=30)
oof_knn, prediction_knn, scores_knn = train_model(X=X_train_scaled,
                                                  X_test=X_test_scaled,
                                                  y=y_tr,
                                                  folds=folds_models,
                                                  params=params_knn,
                                                  model_type='sklearn',
                                                  model=model,
                                                  show_scatter=False)
del model

### GradientBoostingRegressor

In [ ]:
params_gbr = {
    'max_depth': 12,
    'max_features': 'sqrt',
    'min_samples_leaf': 2,
    'min_samples_split': 4
}
model = GradientBoostingRegressor(criterion='mae', n_estimators=200, **params_gbr)
oof_gbr, prediction_gbr, scores_gbr = train_model(X=X_train_scaled,
                                                  X_test=X_test_scaled,
                                                  y=y_tr,
                                                  folds=folds_models,
                                                  params=params_gbr,
                                                  model_type='sklearn',
                                                  model=model,
                                                  show_scatter=False)

Fold 0 started at Fri May 10 17:41:17 2019


# Compare models

In [ ]:
plt.figure(figsize=(12, 8));
scores_df = pd.DataFrame()
try:
    scores_df['LGBMRegressor'] = scores_lgb
except NameError:
    print('LGBMRegressor not computed')
try:
    scores_df['XGBRegressor'] = scores_xgb
except NameError:
    print('XGBRegressor not computed')
try:
    scores_df['NuSVR'] = scores_svr
except NameError:
    print('NuSVR not computed')
try:
    scores_df['CatBoostRegressor'] = scores_cat
except NameError:
    print('CatBoostRegressor not computed')
try:
    scores_df['Ridge'] = scores_r
except NameError:
    print('Ridge not computed')
try:
    scores_df['GPI'] = scores_gpi
except NameError:
    print('GPI not computed')
try:
    scores_df['RandomForestRegressor'] = scores_rf
except NameError:
    print('RandomForestRegressor not computed')
try:
    scores_df['ExtraTreesRegressor'] = scores_ex
except NameError:
    print('ExtraTreesRegressor not computed')
try:
    scores_df['AdaBoostRegressor'] = scores_ada
except NameError:
    print('AdaBoostRegressor not computed')
try:
    scores_df['KNN'] = scores_knn
except NameError:
    print('KNN not computed')
try:
    scores_df['GradientBoostingRegressor'] = scores_knn
except NameError:
    print('GradientBoostingRegressor not computed')

ax = sns.boxplot(data=scores_df.reindex(scores_df.mean().sort_values().index, axis=1));
#ax.set(yscale="log")

plt.xticks(rotation=45);
plt.xlabel('Method');
plt.ylabel('Score');

## Feature reduction

### ELI5 and permutation importance
ELI5 is a package with provides explanations for ML models. It can do this not only for linear models, but also for tree based like Random Forest or lightgbm.

**Important notice**: running eli5 on all features takes a lot of time, so I run the cell below in `version 14` and printed the top-50 features. In the following versions I'll use these 50 columns and use eli5 to find top-40 of them so that it takes less time

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_scaled, y_tr, test_size=0.2)
model = lgb.LGBMRegressor(**params_lgb, n_estimators = 50000, n_jobs = -1, verbose=-1)
model.fit(X_train,
          y_train, 
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
          eval_metric='mae',
          verbose=10000,
          early_stopping_rounds=200)

perm = eli5.sklearn.PermutationImportance(model, random_state=1).fit(X_train, y_train)
eli5.show_weights(perm, top=50, feature_names=X_train_scaled.columns.values.tolist())

In [ ]:
top_num_features = 40
top_features = [i for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if 'BIAS' not in i][:top_num_features]
oof_lgb_top, prediction_lgb_top, scores_lgb_top, feature_importance_lgb_top = train_model(
    X=X_train_scaled[top_features],
    X_test=X_test_scaled[top_features],
    y=y_tr,
    folds=folds_models,
    params=params_lgb,
    model_type='lgb',
    show_scatter=False)

### Dropping highly correlated features

Due to the huge number of features there are certainly some highly correlated features, let's try droping them!

In [ ]:
X_train_scaled_uncorr = X_train_scaled.copy()
X_test_scaled_uncorr = X_test_scaled.copy()

In [ ]:
# https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/
corr_matrix = X_train_scaled_uncorr.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Find index of feature columns with correlation greater than 0.99
to_drop = [column for column in upper.columns if any(upper[column] > 0.99)]
X_train_scaled_uncorr = X_train_scaled_uncorr.drop(to_drop, axis=1)
X_test_scaled_uncorr = X_test_scaled_uncorr.drop(to_drop, axis=1)

print(X_train_scaled.shape)
print(X_train_scaled_uncorr.shape)

In [ ]:
oof_lgb_uncorr, prediction_lgb_uncorr, scores_lgb_uncorr, feature_importance_lgb_uncorr = train_model(
    X=X_train_scaled_uncorr,
    X_test=X_test_scaled_uncorr,
    y=y_tr,
    folds=folds_models,
    params=params_lgb,
    model_type='lgb',
    show_scatter=False)
cols = feature_importance_lgb_uncorr[['feature', 'importance']].groupby('feature').mean().sort_values(
    by='importance', ascending=False).index
best_features = feature_importance_lgb_uncorr.loc[feature_importance_lgb_uncorr.feature.isin(cols)]
plt.figure(figsize=(16, 20));
sns.barplot(x='importance', y='feature', data=best_features.sort_values(by='importance', ascending=False));
plt.title('LGB Features (avg over folds)');

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_scaled_uncorr, y_tr, test_size=0.2)
model = lgb.LGBMRegressor(**params_lgb, n_estimators = 50000, n_jobs = -1, verbose=-1)
model.fit(X_train,
          y_train, 
          eval_set=[(X_train, y_train), (X_valid, y_valid)],
          eval_metric='mae',
          verbose=10000,
          early_stopping_rounds=200)

perm = eli5.sklearn.PermutationImportance(model, random_state=1).fit(X_train, y_train)
#eli5.show_weights(perm, top=50, feature_names=X_train_scaled_uncorr.columns.values.tolist())

In [ ]:
params_lgb = {'num_leaves': 36,
              'min_data_in_leaf': 20,
              'max_depth': 4,
              'learning_rate': 0.005,
              'bagging_freq': 5,
              'bagging_fraction': 0.6,
              'reg_alpha': 0.3833333333333333,
              'reg_lambda': 0.5722222222222222,
              'objective': 'huber',
              'boosting': 'gbdt',
              'verbosity': -1, 'random_seed': 19,
              'metric': 'mae',
              'bagging_seed': 11}
top_num_features = 40
top_features = [i for i in eli5.formatters.as_dataframe.explain_weights_df(model).feature if 'BIAS' not in i][:top_num_features]
oof_lgb_uncorr_top, prediction_lgb_uncorr_top, scores_lgb_uncorr_top, feature_importance_lgb_uncorr_top = train_model(
    X=X_train_scaled_uncorr[top_features],
    X_test=X_test_scaled_uncorr[top_features],
    y=y_tr,
    folds=folds_models,
    params=params_lgb,
    model_type='lgb',
    show_scatter=False)

In [ ]:
evaluation = pd.DataFrame(dtype=np.float64)
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
submission['time_to_failure'] = prediction_lgb_uncorr_top
submission.to_csv('../output/submission_lgb_uncorr_top.csv')

In [ ]:
fixed_params = {
    'objective': 'huber',
    'boosting': 'gbdt',
    'verbosity': -1,
    'random_seed': 19,
    'n_estimators': 50000,
    'metric': 'mae',
    'bagging_seed': 11
}

param_grid = {
    'num_leaves': list(range(8, 92, 4)),
    'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [3, 4, 5, 6, 8, 12, 16, -1],
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'bagging_freq': [3, 4, 5, 6, 7],
    'bagging_fraction': np.linspace(0.6, 0.95, 10),
    'reg_alpha': np.linspace(0.1, 0.95, 10),
    'reg_lambda': np.linspace(0.1, 0.95, 10)
}

grid_size = 1
for param in param_grid:
    grid_size *= len(param_grid[param])
print(f'The search grid has {grid_size} elements')

best_score = 9999
dataset = lgb.Dataset(X_train_scaled_uncorr[top_features], label=y_tr)  # no need to scale features

scores_val_mean = []
scores_val_std = []
for i in tqdm_notebook(range(2000)):
    params = {k: random.choice(v) for k, v in param_grid.items()}
    params.update(fixed_params)
    result = lgb.cv(params,
                    dataset,
                    nfold=n_fold,
                    early_stopping_rounds=200,
                    stratified=False)

    print(
        f"Iteration {i} finished with mae={result['l1-mean'][-1]:.4f} and std={result['l1-stdv'][-1]:.4f}")
    scores_val_mean.append(result['l1-mean'][-1])
    scores_val_std.append(result['l1-stdv'][-1])

    if result['l1-mean'][-1] < best_score:
        best_score = result['l1-mean'][-1]
        best_score_std = result['l1-stdv'][-1]
        best_params = params


plt.figure(figsize=(16, 6))
plt.scatter(scores_val_mean, scores_val_std, color='blue')
plt.scatter(best_score, best_score_std, color='gold')
plt.xlabel('scores_val_mean')
plt.ylabel('scores_val_std')
plt.title('Validation score mean/std scatter plot')
plt.grid()
plt.legend(['All parameters', 'Best'])

print(f"best_score={best_score}")
print(best_params)

In [ ]:
plt.figure(figsize=(12, 8));
scores_df = pd.DataFrame()

try:
    scores_df['LGB'] = scores_lgb
except NameError:
    print('LGB not computed')
try:
    scores_df['LGB_top'] = scores_lgb_top
except NameError:
    print('LGB_top not computed')
try:
    scores_df['LGB_uncorr'] = scores_lgb_uncorr
except NameError:
    print('LGB_uncorr not computed')
try:
    scores_df['LGB_uncorr_top'] = scores_lgb_uncorr_top
except NameError:
    print('LGB_uncorr_top not computed')
try:
    scores_df['XGB'] = scores_xgb
except NameError:
    print('XGB not computed')
try:
    scores_df['SVR'] = scores_svr
except NameError:
    print('SVR not computed')
try:
    scores_df['CatBoost'] = scores_cat
except NameError:
    print('CatBoost not computed')
try:
    scores_df['Ridge'] = scores_r
except NameError:
    print('Ridge not computed')
try:
    scores_df['GPI'] = scores_gpi
except NameError:
    print('GPI not computed')
try:
    scores_df['GPI_new'] = scores_gpi_new
except NameError:
    print('GPI_new not computed')
try:
    scores_df['GPII_new'] = scores_gpii_new
except NameError:
    print('GPII_new not computed')    
try:
    scores_df['GPIII_new'] = scores_gpiii_new
except NameError:
    print('GPIII_new not computed')     
try:
    scores_df['RandomForest'] = scores_rf
except NameError:
    print('RandomForest not computed')     
try:
    scores_df['ExtraTrees'] = scores_ex
except NameError:
    print('ExtraTrees not computed') 
try:
    scores_df['KNN'] = scores_knn
except NameError:
    print('KNN not computed') 
try:
    scores_df['GBR'] = scores_knn
except NameError:
    print('GBR not computed')
    
ax = sns.boxplot(data=scores_df.reindex(scores_df.mean().sort_values().index, axis=1));
#ax.set(yscale="log")

plt.xticks(rotation=45);
plt.xlabel('Method');
plt.ylabel('Score');

In [ ]:
prediction_blend_gp = (prediction_gpi +
                    prediction_gpi_new + 
                    prediction_gpii_new + 
                    prediction_gpiii_new) / 4
oof_blend_gp = (oof_gpi +
             oof_gpi_new + 
             oof_gpii_new + 
             oof_gpiii_new) / 4

In [ ]:
evaluation = pd.DataFrame(dtype=np.float64)
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
submission['time_to_failure'] = prediction_blend_gp
submission.to_csv('../output/submission_blend_gp.csv')

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(12,5))
ax1, ax2 = axis
ax1.set_xlabel('actual')
ax1.set_ylabel('predicted')
ax2.set_xlabel('train index')
ax2.set_ylabel('time to failure')
ax1.scatter(y_tr, oof_lgb_uncorr_top, color='brown')
ax1.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)], color='blue')
ax2.plot(y_tr, color='blue', label='y_train')
ax2.plot(oof_lgb_uncorr_top, color='orange')
plt.show()

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(12,5))
ax1, ax2 = axis
ax1.set_xlabel('actual')
ax1.set_ylabel('predicted')
ax2.set_xlabel('train index')
ax2.set_ylabel('time to failure')
ax1.scatter(y_tr, oof_blend, color='brown')
ax1.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)], color='blue')
ax2.plot(y_tr, color='blue', label='y_train')
ax2.plot(oof_blend, color='orange')
plt.show()